## Goal 
Test different scenarios

In [1]:
import torch
import onnx
import onnxruntime

from typing import Any, List
import torch.nn as nn


torch.manual_seed(0)


### Some thoughts on eager-mode and JIT

Some links: 
1. https://backtrace.blog/2023/05/21/understanding-pytorch-eager-and-graph-mode/
2. https://www.goldsborough.me/ml/ai/python/2018/02/04/20-17-20-a_promenade_of_pytorch/


#### Torch ()
   * **Eager Mode (Define-by-run)**:  the computational graph of a model is defined on the fly as the operations are run. Eager execution allows for dynamic control flow in the network, including loops, ifs, and other Python control structures. 
   * **Graph Mode (Define-and-run)**: Graph mode, on the other hand, builds the entire computation graph before running the computation. The graph-based execution in PyTorch is provided via **TorchScript**, which uses a Just-In-Time (JIT) compiler to convert PyTorch models to a graph representation. The major advantage of this mode is that it can perform various optimizations to speed up execution, and it allows models to be run in non-Python environments, which is crucial for deployment scenarios.

#### Torch vs TensforFlow w.r.t control flow. 
In a static graph environment of TF, control flow must be represented as specialized nodes in the graph. For example, to enable branching, Tensorflow has a tf.cond() operation, which takes three subgraphs as input: a condition subgraph and two subgraphs for the if and else branches of the conditional. Similarly, loops must be represented in TensorFlow graphs as tf.while() operations, taking a condition and body subgraph as input. This operation done first to construct the graph before running it.
In PyTorch, the computational graph is created dynamically at runtime, when you actually run data through the model. So when you execute output = model(input), the operations defined in your model’s forward() function are performed on the input data, and the computational graph is built as these operations are executed.

#### Question: So, how dynamic control flow is handled in torch?
When you call model(input) again, PyTorch will create a new computational graph. This is because, in PyTorch’s “define-by-run” paradigm, the computational graph is built dynamically each time you perform a forward pass with its new data. 

**Note that** , the graph can potentially be different from the one in the first forward pass, since the operations can be dynamic (e.g., they can depend on Python control flow like if statements or for loops).



## Test Case 1: data dependent control flow


In [2]:
class Foo(torch.nn.Module):
    def __init__(self):
        super(Foo, self).__init__()
    def forward(self, x):
        # It is data dependent and Trace will only work with one path
        if x.min() > 0:
            return torch.square(x) 
        else: 
            return -1 * torch.square(x)

    
#def FoF(x):
#    return torch.sqrt(x) if x.max() > 0 else torch.square(x)

In [3]:
foo = Foo()

In [4]:
sample_tensor = torch.rand(1, 10)

positive_tensor = sample_tensor.clone()
negative_tensor = sample_tensor.clone()

# Make all elements of the positive tensor positive
positive_tensor[positive_tensor < 0] *= -1

# Make all elements of the negative tensor negative
negative_tensor[negative_tensor > 0] *= -1

In [5]:
positive_tensor, negative_tensor

(tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074, 0.6341, 0.4901, 0.8964, 0.4556,
          0.6323]]),
 tensor([[-0.4963, -0.7682, -0.0885, -0.1320, -0.3074, -0.6341, -0.4901, -0.8964,
          -0.4556, -0.6323]]))

In [6]:
positive_output = foo(positive_tensor)
negative_output = foo(negative_tensor)

positive_output, negative_output

(tensor([[0.2463, 0.5902, 0.0078, 0.0174, 0.0945, 0.4021, 0.2402, 0.8036, 0.2076,
          0.3998]]),
 tensor([[-0.2463, -0.5902, -0.0078, -0.0174, -0.0945, -0.4021, -0.2402, -0.8036,
          -0.2076, -0.3998]]))

### tracing
Apparantly tracing does not work.

In tracing, it run the code, record the operations that happen and construct a ScriptModule that does exactly that. Unfortunately, things like control flow are erased.

In [7]:
traced_class = torch.jit.trace(Foo(), positive_output)

/tmp/ipykernel_2053/626634658.py:6: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.min() > 0:


In [8]:
type(traced_class)

torch.jit._trace.TopLevelTracedModule

In [9]:
traced_class.code

'def forward(self,\n    x: Tensor) -> Tensor:\n  return torch.square(x)\n'

In [10]:
print(traced_class.graph)

graph(%self : __torch__.Foo,
      %x : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu)):
  %7 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu) = aten::square(%x) # /tmp/ipykernel_2053/626634658.py:7:0
  return (%7)



In [11]:
traced_class(negative_tensor)

tensor([[0.2463, 0.5902, 0.0078, 0.0174, 0.0945, 0.4021, 0.2402, 0.8036, 0.2076,
         0.3998]])

In [12]:
scripted_class = torch.jit.script(foo)

In [13]:
print(scripted_class.code)

def forward(self,
    x: Tensor) -> Tensor:
  if bool(torch.gt(torch.min(x), 0)):
    _0 = torch.square(x)
  else:
    _0 = torch.mul(torch.square(x), -1)
  return _0



In [14]:
scripted_class(negative_tensor)

tensor([[-0.2463, -0.5902, -0.0078, -0.0174, -0.0945, -0.4021, -0.2402, -0.8036,
         -0.2076, -0.3998]])

In [19]:
torch.onnx.export(scripted_class, positive_tensor, "./toy_models/foo_model_scripted.onnx", verbose=True, opset_version=17)

Exported graph: graph(%x.1 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu)):
  %/ReduceMin_output_0 : Float(device=cpu) = onnx::ReduceMin[keepdims=0, onnx_name="/ReduceMin"](%x.1), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:6:11
  %/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={0}, onnx_name="/Constant"](), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:6:11
  %/Greater_output_0 : Bool(device=cpu) = onnx::Greater[onnx_name="/Greater"](%/ReduceMin_output_0, %/Constant_output_0), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:6:11
  %/Cast_output_0 : Bool(device=cpu) = onnx::Cast[to=9, onnx_name="/Cast"](%/Greater_output_0), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:6:8
  %5 : Float(1, 10, strides=[10, 1], device=cpu) = onnx::If[onnx_name="/If"](%/Cast_output_0), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:6:8
    block0():
      %/Mul_output_0 : Float(1, 10, strides=[10, 1], device=cpu) = onnx::Mul[onnx_name="/Mul"](%x.1, %x

In [20]:
ort_session = onnxruntime.InferenceSession("./toy_models/foo_model_scripted.onnx")

# Run inference with sample input
ort_input_value = negative_tensor.detach().numpy()
ort_inputs = {ort_session.get_inputs()[0].name: ort_input_value}
ort_outputs = ort_session.run(None, ort_inputs)


In [21]:
ort_outputs

[array([[-0.2462706 , -0.5901647 , -0.00782826, -0.01743205, -0.09450879,
         -0.40205577, -0.24019155, -0.8036132 , -0.20759685, -0.39981124]],
       dtype=float32)]

**observation**: onnx on scripted class works but fails on traced class.

In [22]:
torch.onnx.export(traced_class, positive_tensor, "./toy_models/foo_model_traced.onnx", verbose=True, opset_version=17)

Exported graph: graph(%x : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu)):
  %1 : Float(1, 10, strides=[10, 1], requires_grad=0, device=cpu) = onnx::Mul[onnx_name="/Mul"](%x, %x), scope: Foo:: # /tmp/ipykernel_2053/626634658.py:7:0
  return (%1)



/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:823: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f150eb16270>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


In [24]:
ort_session = onnxruntime.InferenceSession("./toy_models/foo_model_traced.onnx")

# Run inference with sample input
ort_input_value = negative_tensor.detach().numpy()
ort_inputs = {ort_session.get_inputs()[0].name: ort_input_value}
ort_outputs = ort_session.run(None, ort_inputs)
ort_outputs

[array([[0.2462706 , 0.5901647 , 0.00782826, 0.01743205, 0.09450879,
         0.40205577, 0.24019155, 0.8036132 , 0.20759685, 0.39981124]],
       dtype=float32)]

In [ ]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        # Define your layers here
        self.linear = torch.nn.Linear(10, 5)
        self.relu = torch.nn.ReLU()

    def forward(self, x_tensor, y_integer):
        # Your forward pass logic here
        x = self.linear(x_tensor)
        x = self.relu(x)
        x = x * y_integer  # You can perform operations with the integer input
        return x

# Instantiate your model
model = MyModule()

# Example input
input_tensor = torch.randn(1, 10)
input_integer = 3

# Trace the model
traced_model = torch.jit.trace(model, (input_tensor, input_integer))

In [ ]:
def adapter(inputs):
    """
    Convert inputs to tensors if they are not already tensors.

    Args:
    inputs (list): List of inputs.

    Returns:
    list: List of tensors.
    """
    tensor_inputs = []
    for item in inputs:
        if not isinstance(item, torch.Tensor):
            item = torch.tensor(item)
        tensor_inputs.append(item)
    return tuple(tensor_inputs)

# Example usage:
input_list = [torch.randn(1, 10), 1]
output_list = adapter(input_list)
print("Output List of Tensors:", output_list)

In [ ]:
torch.jit.trace(model, adapter(input_list))

In [ ]:
model(*adapter(input_list))

In [ ]:
x0, x1 = adapter(input_list)

In [ ]:
# Instantiate the model
model = Foo()

# Create sample input tensor
small_tensor = torch.rand(1, 10) * 0.49
print("small_tensor max is smaller than threshold: ", small_tensor.max() < 0.5)

large_tensor = torch.rand(1, 10) + 0.5
print("large_tensor max is smaller than threshold: ", large_tensor.max() < 0.5)



In [ ]:
traced_model = torch.jit.trace(Foo(), small_tensor)

In [ ]:
print(traced_model.code)

In [ ]:
scripted_model = torch.jit.script(model)

In [ ]:
print(scripted_model.code)

In [ ]:
torch.onnx.export(model, small_tensor, "foo_model.onnx", verbose=True, opset_version=17)

In [ ]:
torch.onnx.export(traced_model, small_tensor, "foo_model_traced.onnx", verbose=True, opset_version=17)

In [ ]:
torch.onnx.export(scripted_model, small_tensor, "foo_model_scripted.onnx", verbose=True, opset_version=17)

In [ ]:
from typing import Any, List
import torch

class TestModule(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input: torch.Tensor) -> Any:
        ret: List[torch.Tensor] = []
        if input.shape[0] == 1:
            return input
        else:
            ret.append(input)
            return ret

m = TestModule()
m_scripted = torch.jit.script(m)

In [ ]:
print(m_scripted.code)

In [ ]:
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = torch.nn.ModuleList([
            torch.nn.Linear(32, 32),
            torch.nn.Linear(32, 32)
        ])

    def forward(self, x, n_layers: int):
        for i in range(n_layers):
            x = self.layers[i](x)
        return x

torch.jit.script(Model())

In [ ]:
from torch.nn import ModuleList
import torch




class MyEncoderLayer(torch.nn.Module):
    def __init__(self, d=32):
        super().__init__()
        self.d = d
        self.empty = False
        self.layer = torch.nn.Linear(d, d)
    
    def forward(self, x: torch.Tensor):
        if torch.sum(x) > 0:
            return self.layer(x)
        return self.layer(2 * x)

class MyModule(torch.nn.Module):
    def __init__(self, d):
        super().__init__()
        self.d = d
        self.empty = False
        self.layers = torch.nn.ModuleList([MyEncoderLayer(32), MyEncoderLayer(32)])

    def forward(self, x: torch.Tensor, c: int):
        result = torch.rand(self.d)
        for i, submodule in enumerate(self.layers):
            if i < c and not submodule.empty:
                result = submodule.forward(x)
        return result 

d = 32
script = torch.jit.script(MyModule(d))
out = script.forward(torch.randn(d), 2)

In [ ]:
def forward(self, x: torch.Tensor, c: int):
        result = torch.rand(self.d)
        for i, submodule in enumerate(self.layers):
            if i < c and not submodule.empty:
                result = submodule.forward(x)
        return result 

In [ ]:
import torch
import onnx
import onnxruntime


@torch.jit.script
def status(x):
    if x.max() > 0.5:
        return True
    else:
        return False



class Foo(torch.nn.Module):
    def __init__(self):
        super(Foo, self).__init__()
    def forward(self, tensor):
        # It is data dependent and Trace will only work with one path
        if status(tensor):
            return tensor ** 2
        return tensor

# Instantiate the model
model = Foo()

# Create sample input tensor
small_tensor = torch.rand(1, 10) * 0.5
print("small_tensor max is smaller than threshold: ", small_tensor.max() < 0.5)

large_tensor = torch.rand(1, 10) + 0.5
print("large_tensor max is smaller than threshold: ", large_tensor.max() < 0.5)



In [ ]:
model(small_tensor)

In [ ]:
scripted_Foo = torch.jit.script(Foo())

In [ ]:
print(scripted_Foo.code)

In [ ]:
scripted_Foo(small_tensor)

In [ ]:
import torch
import onnx
import onnxruntime

@torch.jit.export
def status(x):
    if x.max() > 0.5:
        return True
    else:
        return False



class Foo(torch.nn.Module):
    def __init__(self):
        super(Foo, self).__init__()
    def forward(self, tensor):
        # It is data dependent and Trace will only work with one path
        if status(tensor):
            return tensor ** 2
        return tensor

# Instantiate the model
model = Foo()

# Create sample input tensor
small_tensor = torch.rand(1, 10) * 0.5
print("small_tensor max is smaller than threshold: ", small_tensor.max() < 0.5)

large_tensor = torch.rand(1, 10) + 0.5
print("large_tensor max is smaller than threshold: ", large_tensor.max() < 0.5)


In [ ]:
scripted_Foo = torch.jit.script(Foo())

In [ ]:
scripted_Foo(small_tensor)

In [ ]:
small_tensor_output = model(small_tensor)
small_tensor_output

In [ ]:
large_tensor_output = model(large_tensor)
large_tensor_output

In [ ]:
# Trace the model using example input
#traced_model = torch.jit.trace(model, small_tensor)
if False:
    # Export the traced model to ONNX
    torch.onnx.export(model, small_tensor, "foo_model.onnx", verbose=True, opset_version=17)

    # Load the ONNX model using ONNX Runtime
    ort_session = onnxruntime.InferenceSession("foo_model.onnx")

    # Run inference with sample input
    ort_input_value = small_tensor.detach().numpy()
    ort_inputs = {ort_session.get_inputs()[0].name: ort_input_value}
    ort_outputs = ort_session.run(None, ort_inputs)

    print("Output:", ort_outputs)

In [ ]:
if True:
    # Export the traced model to ONNX
    torch.onnx.export(model, small_tensor, "l_foo_model.onnx", verbose=True, opset_version=17)

    # Load the ONNX model using ONNX Runtime
    ort_session = onnxruntime.InferenceSession("l_foo_model.onnx")

    # Run inference with sample input
    ort_input_value = large_tensor.detach().numpy()
    ort_inputs = {ort_session.get_inputs()[0].name: ort_input_value}
    ort_outputs = ort_session.run(None, ort_inputs)

    print("Output:", ort_outputs)

In [ ]:
large_tensor_output

In [ ]:
large_tensor_output ** 2

In [ ]:
torch.jit.script(model)

In [ ]:
traced_cell = torch.jit.trace(model, small_tensor)

In [ ]:
import onnx
import onnx.numpy_helper as np_helper
import numpy as np

from onnx import helper, shape_inference

# Define the input tensor shape
input_shape = (10,)

# Create the input tensor and maximum value
input_tensor = np.random.rand(*input_shape).astype(np.float32)
max_value = np.max(input_tensor)

# Check if the maximum value is greater than 0.5
output_value = max_value if max_value > 0.5 else None

# Create the input and output names
input_name = "input"
output_name = "output"

# Create the input tensor value info
input_tensor_value_info = helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, input_shape)

# Create the output tensor value info
output_tensor_value_info = helper.make_empty_tensor_value_info(output_name)

# Create the conditional node
if output_value is not None:
    constant_node = helper.make_node(
        "Constant",
        inputs=[],
        outputs=["condition"],
        value=np_helper.from_array(np.array([output_value], dtype=np.float32)),
    )

    greater_node = helper.make_node(
        "Greater",
        inputs=[input_name, "condition"],
        outputs=[output_name],
    )
else:
    # If maximum value is not greater than 0.5, output should be omitted
    output_tensor_value_info = None

# Create the graph
if output_tensor_value_info is not None:
    graph = helper.make_graph(
        [constant_node, greater_node],
        "max_value_graph",
        [input_tensor_value_info],
        [output_tensor_value_info],
    )
else:
    graph = helper.make_graph(
        [],
        "max_value_graph",
        [input_tensor_value_info],
        [],
    )

# Perform shape inference
inferred_graph = shape_inference.infer_shapes(graph)

# Create the model
onnx_model = helper.make_model(inferred_graph)

# Save the ONNX model to a file
onnx.save(onnx_model, "tpy_model.onnx")

print("ONNX model saved successfully!")

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Generate input tensor
input_tensor = np.random.rand(10).astype(np.float32)

# Run inference
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
output = ort_session.run([output_name], {input_name: input_tensor.reshape(1, -1)})


In [ ]:
import onnx
import onnxruntime
import numpy as np

# Define the input shape
input_shape = (1, 10)

# Create the input tensor with random values
input_data = np.random.rand(*input_shape).astype(np.float32)

# Create the maximum value threshold
threshold = np.array([0.5], dtype=np.float32)

# Create the ONNX model
node_input = onnx.helper.make_tensor_value_info('input', onnx.TensorProto.FLOAT, input_shape)
node_threshold = onnx.helper.make_tensor_value_info('threshold', onnx.TensorProto.FLOAT, [1])
node_output = onnx.helper.make_tensor_value_info('output', onnx.TensorProto.FLOAT, [1])

nodes = [
    onnx.helper.make_node('Max', inputs=['input'], outputs=['max_value']),
    onnx.helper.make_node('Greater', inputs=['max_value', 'threshold'], outputs=['greater_than_threshold']),
    onnx.helper.make_node('Cast', inputs=['greater_than_threshold'], outputs=['cast_output'], to=9),  # Cast to boolean (9)
    onnx.helper.make_node('Where', inputs=['cast_output', 'max_value', 'threshold'], outputs=['output'])
]

graph_def = onnx.helper.make_graph(nodes, 'max_value_graph', [node_input, node_threshold], [node_output])

model_def = onnx.helper.make_model(graph_def, producer_name='max_value_model')

# Save the ONNX model to a file
onnx.save(model_def, 'max_value_model.onnx')

# Run inference using ONNX Runtime
ort_session = onnxruntime.InferenceSession('max_value_model.onnx')
outputs = ort_session.run(None, {'input': input_data, 'threshold': threshold})

print("Output:", outputs)

In [ ]:
import torch

test_input = torch.randn(10)

class Foo(torch.nn.Module):
    def forward(self, tensor):
        # It is data dependent
        # Trace will only work with one path
        if tensor.max() > 0.5:
            return tensor ** 2
        return tensor


model = Foo()
model_scripted_v1 = torch.jit.script(model) # No warnings
model_scripted_v2 = torch.jit.trace(model, test_input) # Warning

In [ ]:
input_data.shape

In [ ]:
input_data = np.random.rand(*input_shape).astype(np.float32)

In [ ]:
ort_session = onnxruntime.InferenceSession('max_value_model.onnx')
outputs = ort_session.run(None, {'input': input_data, 'threshold': threshold})


In [ ]:
input_data

In [ ]:
data_0 = np.array([3, 2, 1]).astype(np.float32)
data_1 = np.array([1, 4, 4]).astype(np.float32)
data_2 = np.array([2, 5, 3]).astype(np.float32)
result = np.array([3, 5, 4]).astype(np.float32)
node = onnx.helper.make_node(
    "Max",
    inputs=["data_0", "data_1", "data_2"],
    outputs=["result"],
)
expect(
    node,
    inputs=[data_0, data_1, data_2],
    outputs=[result],
    name="test_max_example",
)

node = onnx.helper.make_node(
    "Max",
    inputs=["data_0"],
    outputs=["result"],
)
expect(node, inputs=[data_0], outputs=[data_0], name="test_max_one_input")

result = np.maximum(data_0, data_1)
node = onnx.helper.make_node(
    "Max",
    inputs=["data_0", "data_1"],
    outputs=["result"],
)

In [ ]:
from typing import Any, Sequence
import numpy as np
import onnx
import onnxruntime
import subprocess
import sys
from copy import deepcopy
from pathlib import Path
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union

import numpy as np

import onnx
from onnx.backend.test.case.test_case import TestCase
from onnx.backend.test.case.utils import import_recursive
from onnx.onnx_pb import (
    AttributeProto,
    FunctionProto,
    GraphProto,
    ModelProto,
    NodeProto,
    TensorProto,
    TypeProto,
)

_NodeTestCases = []
_TargetOpType = None
_DiffOpTypes = None


def _extract_value_info(
    input: Union[List[Any], np.ndarray, None],
    name: str,
    type_proto: Optional[TypeProto] = None,
) -> onnx.ValueInfoProto:
    if type_proto is None:
        if input is None:
            raise NotImplementedError(
                "_extract_value_info: both input and type_proto arguments cannot be None."
            )
        elif isinstance(input, list):
            elem_type = onnx.helper.np_dtype_to_tensor_dtype(input[0].dtype)
            shape = None
            tensor_type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)
            type_proto = onnx.helper.make_sequence_type_proto(tensor_type_proto)
        elif isinstance(input, TensorProto):
            elem_type = input.data_type
            shape = tuple(input.dims)
            type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)
        else:
            elem_type = onnx.helper.np_dtype_to_tensor_dtype(input.dtype)
            shape = input.shape
            type_proto = onnx.helper.make_tensor_type_proto(elem_type, shape)

    return onnx.helper.make_value_info(name, type_proto)

def expect(
    node: onnx.NodeProto,
    inputs: Sequence[np.ndarray],
    outputs: Sequence[np.ndarray],
    name: str,
    **kwargs: Any,
) -> None:
    # Builds the model
    present_inputs = [x for x in node.input if (x != "")]
    present_outputs = [x for x in node.output if (x != "")]
    input_type_protos = [None] * len(inputs)
    if "input_type_protos" in kwargs:
        input_type_protos = kwargs["input_type_protos"]
        del kwargs["input_type_protos"]
    output_type_protos = [None] * len(outputs)
    if "output_type_protos" in kwargs:
        output_type_protos = kwargs["output_type_protos"]
        del kwargs["output_type_protos"]
    inputs_vi = [
        _extract_value_info(arr, arr_name, input_type)
        for arr, arr_name, input_type in zip(inputs, present_inputs, input_type_protos)
    ]
    outputs_vi = [
        _extract_value_info(arr, arr_name, output_type)
        for arr, arr_name, output_type in zip(
            outputs, present_outputs, output_type_protos
        )
    ]
    graph = onnx.helper.make_graph(
        nodes=[node], name=name, inputs=inputs_vi, outputs=outputs_vi
    )
    kwargs["producer_name"] = "backend-test"

    if "opset_imports" not in kwargs:
        # To make sure the model will be produced with the same opset_version after opset changes
        # By default, it uses since_version as opset_version for produced models
        produce_opset_version = onnx.defs.get_schema(
            node.op_type, domain=node.domain
        ).since_version
        kwargs["opset_imports"] = [
            onnx.helper.make_operatorsetid(node.domain, produce_opset_version)
        ]

    model = onnx.helper.make_model_gen_version(graph, **kwargs)

    # Checking the produces are the expected ones.
    sess = onnxruntime.InferenceSession(model.SerializeToString(),
                                        providers=["CPUExecutionProvider"])
    feeds = {name: value for name, value in zip(node.input, inputs)}
    results = sess.run(None, feeds)
    for expected, output in zip(outputs, results):
        return (results, outputs)
        return np.testing.assert_allclose(expected, output)

In [ ]:
data_0 = np.array([3, 2, 1]).astype(np.float32)
data_1 = np.array([1, 4, 4]).astype(np.float32)
data_2 = np.array([2, 5, 3]).astype(np.float32)
result = np.array([3, 5, 4]).astype(np.float32)
node = onnx.helper.make_node(
    "Max",
    inputs=["data_0", "data_1", "data_2"],
    outputs=["result"],
)
expect(
    node,
    inputs=[data_0, data_1, data_2],
    outputs=[result],
    name="test_max_example",
)

node = onnx.helper.make_node(
    "Max",
    inputs=["data_0"],
    outputs=["result"],
)
expect(node, inputs=[data_0], outputs=[data_0], name="test_max_one_input")

result = np.maximum(data_0, data_1)
node = onnx.helper.make_node(
    "Max",
    inputs=["data_0", "data_1"],
    outputs=["result"],
)

In [ ]:
expect(
    node, inputs=[data_0, data_1], outputs=[result], name="test_max_two_inputs"
)

In [ ]:
import torch
import onnx
import onnxruntime

class Foo(torch.nn.Module):
    def forward(self, tensor):
        # It is data dependent
        # Trace will only work with one path
        if tensor.max() < 0.5:
            return tensor ** 2
        return tensor

# Instantiate the model
model = Foo()

# Create sample input tensor
small_tensor = torch.rand(1, 10) * 0.5

large_tensor = torch.rand(1, 10) + 0.5
print(large_tensor)

# Trace the model using example input
#traced_model = torch.jit.trace(model, small_tensor)

# Export the traced model to ONNX
torch.onnx.export(model, small_tensor, "foo_model.onnx", verbose=True, opset_version=11)

# Load the ONNX model using ONNX Runtime
ort_session = onnxruntime.InferenceSession("foo_model.onnx")

# Run inference with sample input
ort_inputs = {ort_session.get_inputs()[0].name: large_tensor.detach().numpy()}
ort_outputs = ort_session.run(None, ort_inputs)

print("Output:", ort_outputs)

In [ ]:
large_tensor

In [ ]:
small_tensor

In [ ]:
small_tensor

In [ ]:
# Create a tensor of shape (1, 10) with random values between 0 and 1
tensor = torch.rand(1, 10)

# Multiply the tensor by 0.5 to ensure all elements are smaller than 0.5
tensor *= 0.5

In [ ]:
# Run inference with sample input
ort_inputs = {ort_session.get_inputs()[0].name: tensor.detach().numpy()}
ort_outputs = ort_session.run(None, ort_inputs)

print("Output:", ort_outputs)

In [ ]:
import math
node = onnx.helper.make_node(
    "Erf",
    inputs=["x"],
    outputs=["y"],
)

x = np.random.randn(1, 3, 32, 32).astype(np.float32)
y = np.vectorize(math.erf)(x).astype(np.float32)
expect(node, inputs=[x], outputs=[y], name="test_erf")

In [ ]:
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))

    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h

rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

In [ ]:
torch.jit.script(Foo()).code

In [ ]:
x, h = torch.rand(3, 4), torch.rand(3, 4)
print(scripted_cell(x, h))